# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [2]:
# Your code here
# cur.execute('''SELECT (e.firstName + e.lastName) as name
#             FROM employees e
#             JOIN offices o
#             USING(officecode)
#             WHERE o.city IS 'Boston';''')
cur.execute('''SELECT officeCode, city
            FROM offices o
            JOIN employees e
            USING(officecode)
            WHERE o.city IS 'Boston';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,officeCode,city
0,2,Boston
1,2,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [5]:
# Your code here
cur.execute('''SELECT o.officeCode, o.city, COUNT(employeeNumber) as n_employees
            FROM offices o
            LEFT JOIN employees e
            USING(officeCode)
            GROUP BY officeCode
            HAVING n_employees IS 0;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df


,officeCode,city,n_employees
0,27,Boston,0


In [20]:
cur.execute('''SELECT o.officeCode, o.city, employeeNumber
            FROM offices o
            LEFT JOIN employees e
            USING(officeCode);''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.groupby('officeCode').count()

,city,employeeNumber
officeCode,,
1,6,6
2,2,2
3,2,2
4,5,5
5,2,2
6,4,4
7,2,2
27,1,0


## Write 3 Questions of your own and answer them

In [37]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
df = make_df(cur.execute('''SELECT (firstName || ' ' || lastName) as e_name, c.customerNumber, p.productCode, pl.textDescription
                        FROM employees e
                        LEFT JOIN customers c 
                        ON e.employeeNumber = c.salesRepEmployeeNumber
                        LEFT JOIN orders o
                        USING(customerNumber)
                        LEFT JOIN orderdetails od
                        USING(orderNumber)
                        LEFT JOIN products p
                        USING(productCode)
                        LEFT JOIN productLines pl
                        USING(productLine)
                        WHERE c.customerName IS NOT NULL;'''))
df

,e_name,customerNumber,productCode,textDescription
0,Gerard Hernandez,103,S18_1589,Attention car enthusiasts: Make your wildest c...
1,Gerard Hernandez,103,S18_2870,Attention car enthusiasts: Make your wildest c...
2,Gerard Hernandez,103,S18_3685,Attention car enthusiasts: Make your wildest c...
3,Gerard Hernandez,103,S24_1628,Attention car enthusiasts: Make your wildest c...
4,Gerard Hernandez,103,S10_2016,Our motorcycles are state of the art replicas ...
...,...,...,...,...
2993,Peter Marsh,496,S12_2823,Our motorcycles are state of the art replicas ...
2994,Peter Marsh,496,S18_2625,Our motorcycles are state of the art replicas ...
2995,Peter Marsh,496,S24_1578,Our motorcycles are state of the art replicas ...
2996,Peter Marsh,496,S24_2000,Our motorcycles are state of the art replicas ...


In [31]:
df = make_df(cur.execute('''SELECT * FROM productLines;'''))
df

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,,
1,Motorcycles,Our motorcycles are state of the art replicas ...,,
2,Planes,"Unique, diecast airplane and helicopter replic...",,
3,Ships,The perfect holiday or anniversary gift for ex...,,
4,Trains,Model trains are a rewarding hobby for enthusi...,,
5,Trucks and Buses,The Truck and Bus models are realistic replica...,,
6,Vintage Cars,Our Vintage Car models realistically portray a...,,


In [43]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
df = make_df(cur.execute("""SELECT textDescription, firstName, lastName
                            FROM productlines
                            JOIN products
                            USING(productLine)
                            JOIN orderdetails
                            USING(productCode)
                            JOIN orders
                            USING(orderNumber)
                            JOIN customers
                            USING(customerNumber)
                            JOIN employees
                            ON(employees.employeeNumber = customers.salesRepEmployeeNumber);"""))
df

,textDescription,firstName,lastName
0,Attention car enthusiasts: Make your wildest c...,Barry,Jones
1,Attention car enthusiasts: Make your wildest c...,Barry,Jones
2,Attention car enthusiasts: Make your wildest c...,Martin,Gerard
3,Attention car enthusiasts: Make your wildest c...,Leslie,Jennings
4,Attention car enthusiasts: Make your wildest c...,Mami,Nishi
...,...,...,...
2991,Our Vintage Car models realistically portray a...,Gerard,Hernandez
2992,Our Vintage Car models realistically portray a...,Steve,Patterson
2993,Our Vintage Car models realistically portray a...,Loui,Bondur
2994,Our Vintage Car models realistically portray a...,George,Vanauf


In [ ]:
# Your code here

In [ ]:
# Your code here

## Level Up: Display the names of every individual product that each employee has sold

In [21]:
def make_df(data):
    df = pd.DataFrame(data.fetchall())
    df.columns = [x[0] for x in data.description]
    return df

In [45]:
df = make_df(cur.execute('''SELECT (firstName || ' ' || lastName) as e_name, c.customerName, p.productName
                        FROM employees e
                        JOIN customers c 
                        ON e.employeeNumber = c.salesRepEmployeeNumber
                        JOIN orders o
                        USING(customerNumber)
                        JOIN orderdetails od
                        USING(orderNumber)
                        JOIN products p
                        USING(productCode);'''))
df


,e_name,customerName,productName
0,Leslie Jennings,Mini Gifts Distributors Ltd.,1958 Setra Bus
1,Leslie Jennings,Mini Gifts Distributors Ltd.,1940 Ford Pickup Truck
2,Leslie Jennings,Mini Gifts Distributors Ltd.,1939 Cadillac Limousine
3,Leslie Jennings,Mini Gifts Distributors Ltd.,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie Jennings,Mini Gifts Distributors Ltd.,1968 Ford Mustang
...,...,...,...
2991,Martin Gerard,"Iberia Gift Imports, Corp.",1954 Greyhound Scenicruiser
2992,Martin Gerard,"Iberia Gift Imports, Corp.",1950's Chicago Surface Lines Streetcar
2993,Martin Gerard,"Iberia Gift Imports, Corp.",Diamond T620 Semi-Skirted Tanker
2994,Martin Gerard,"Iberia Gift Imports, Corp.",1911 Ford Town Car


In [49]:
df = make_df(cur.execute('''SELECT * FROM employees;'''))
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


In [ ]:
# Your code here
cur.execute('''SELECT (firstName || ' ' || lastName) as name, 

            FROM office o
            JOIN employees e
            USING(officecode)
            WHERE o.city IS 'Boston';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

## Level Up: Display the Number of Products each employee has sold

In [47]:
# Your code here
df = make_df(cur.execute('''SELECT (firstName || ' ' || lastName) as e_name, count(*) as number_of_products_sold
                        FROM employees e
                        JOIN customers c 
                        ON e.employeeNumber = c.salesRepEmployeeNumber
                        JOIN orders o
                        USING(customerNumber)
                        JOIN orderdetails od
                        USING(orderNumber)
                        JOIN products p
                        USING(productCode)
                        GROUP BY e_name
                        ORDER BY count(*) DESC;'''))
df

,e_name,number_of_products_sold
0,Gerard Hernandez,396
1,Leslie Jennings,331
2,Pamela Castillo,272
3,Larry Bott,236
4,Barry Jones,220
5,George Vanauf,211
6,Andy Fixter,185
7,Peter Marsh,185
8,Loui Bondur,177
9,Steve Patterson,152


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!